<a href="https://colab.research.google.com/github/vapurv/Cab-fare-prediction/blob/master/cab_fare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#By Apurv verma
import os 
import tensorflow as tf
import tensorboardcolab as tbd
import datetime
import filecmp
import pandas as pd 
import numpy as np  
import matplotlib.pyplot as plt
import seaborn as sns 
from collections import Counter 
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression #ML algorithm
from sklearn.model_selection import train_test_split #splitting dataset
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from pprint import pprint
from sklearn.model_selection import GridSearchCV    


%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
train  = pd.read_csv("train_cab.csv",na_values={"pickup_datetime":"43"})

In [3]:
train

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1.0
1,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1.0
2,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2.0
3,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1.0
4,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1.0
...,...,...,...,...,...,...,...
16062,6.5,2014-12-12 07:41:00 UTC,-74.008820,40.718757,-73.998865,40.719987,1.0
16063,16.1,2009-07-13 07:58:00 UTC,-73.981310,40.781695,-74.014392,40.715527,2.0
16064,8.5,2009-11-11 11:19:07 UTC,-73.972507,40.753417,-73.979577,40.765495,1.0
16065,8.1,2010-05-11 23:53:00 UTC,-73.957027,40.765945,-73.981983,40.779560,1.0


In [4]:
test_data = pd.read_csv('test.csv')

In [5]:
test_data


,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2015-01-27 13:08:24 UTC,-73.973320,40.763805,-73.981430,40.743835,1
1,2015-01-27 13:08:24 UTC,-73.986862,40.719383,-73.998886,40.739201,1
2,2011-10-08 11:53:44 UTC,-73.982524,40.751260,-73.979654,40.746139,1
3,2012-12-01 21:12:12 UTC,-73.981160,40.767807,-73.990448,40.751635,1
4,2012-12-01 21:12:12 UTC,-73.966046,40.789775,-73.988565,40.744427,1
...,...,...,...,...,...,...
9909,2015-05-10 12:37:51 UTC,-73.968124,40.796997,-73.955643,40.780388,6
9910,2015-01-12 17:05:51 UTC,-73.945511,40.803600,-73.960213,40.776371,6
9911,2015-04-19 20:44:15 UTC,-73.991600,40.726608,-73.789742,40.647011,6
9912,2015-01-31 01:05:19 UTC,-73.985573,40.735432,-73.939178,40.801731,6


In [6]:
train.head() #Head Returns the first n rows.

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1.0
1,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1.0
2,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2.0
3,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1.0
4,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1.0


In [7]:
test_data.head() #This will return first five rows of the test data set.

,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2015-01-27 13:08:24 UTC,-73.973320,40.763805,-73.981430,40.743835,1
1,2015-01-27 13:08:24 UTC,-73.986862,40.719383,-73.998886,40.739201,1
2,2011-10-08 11:53:44 UTC,-73.982524,40.751260,-73.979654,40.746139,1
3,2012-12-01 21:12:12 UTC,-73.981160,40.767807,-73.990448,40.751635,1
4,2012-12-01 21:12:12 UTC,-73.966046,40.789775,-73.988565,40.744427,1


In [8]:
#For the better understanding of data,we will check which type of data are present in the training and test data set.
train.dtypes

fare_amount           object
pickup_datetime       object
pickup_longitude     float64
pickup_latitude      float64
dropoff_longitude    float64
dropoff_latitude     float64
passenger_count      float64
dtype: object

In [9]:
#Checking the type of data in test set.
test_data.dtypes

pickup_datetime       object
pickup_longitude     float64
pickup_latitude      float64
dropoff_longitude    float64
dropoff_latitude     float64
passenger_count        int64
dtype: object

In [10]:
#Checking the number of rows and column in the training data set
print('\t','Number of rows','','Number of colummns')
x,y = train.shape
print('\t',x,'      \t',y)

	 Number of rows  Number of colummns
	 16067       	 7


In [11]:
#Checking the number of rows and column in the test data set
print('\t','Number of rows','','Number of colummns')
x,y = test_data.shape
print('\t',x,'      \t',y)

	 Number of rows  Number of colummns
	 9914       	 6


In [12]:
#Descrbing the data set:Describe(): The describe() method is used for calculating some statistical data like percentile, mean and std of the numerical values of the Series or DataFrame. It analyzes both numeric and object series and also the DataFrame column sets of mixed data types.
test_data.describe()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,9914.000000,9914.000000,9914.000000,9914.000000,9914.000000
mean,-73.974722,40.751041,-73.973657,40.751743,1.671273
std,0.042774,0.033541,0.039072,0.035435,1.278747
min,-74.252193,40.573143,-74.263242,40.568973,1.000000
25%,-73.992501,40.736125,-73.991247,40.735254,1.000000
50%,-73.982326,40.753051,-73.980015,40.754065,1.000000
75%,-73.968013,40.767113,-73.964059,40.768757,2.000000
max,-72.986532,41.709555,-72.990963,41.696683,6.000000


In [13]:
#From the description obatined -values are found are percentile at 25%,50% and 75%.Mean std and minimum are also obtained.

In [14]:
#Descrbing the data set:Describe(): The describe() method is used for calculating some statistical data like percentile, mean and std of the numerical values of the Series or DataFrame. It analyzes both numeric and object series and also the DataFrame column sets of mixed data types.
train.describe()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,16067.000000,16067.000000,16067.000000,16067.000000,16012.000000
mean,-72.462787,39.914725,-72.462328,39.897906,2.625070
std,10.578384,6.826587,10.575062,6.187087,60.844122
min,-74.438233,-74.006893,-74.429332,-74.006377,0.000000
25%,-73.992156,40.734927,-73.991182,40.734651,1.000000
50%,-73.981698,40.752603,-73.980172,40.753567,1.000000
75%,-73.966838,40.767381,-73.963643,40.768013,2.000000
max,40.766125,401.083332,40.802437,41.366138,5345.000000


**Data Cleaning and missing value**

**On check of the types of data available in the data set we found that cab fare and pickup date time are of object data type.So it need to be converted to numeric**

In [15]:
#Convert fare_amount from object to numeric
train["fare_amount"] = pd.to_numeric(train["fare_amount"],errors = "coerce") 
 #Using errors=’coerce’. It will replace all non-numeric values with NaN.

**Checking the train data type again(fare amount must be convverted to numeric data type)**

In [53]:
train.dtypes


fare_amount                 float64
pickup_datetime      datetime64[ns]
pickup_longitude            float64
pickup_latitude             float64
dropoff_longitude           float64
dropoff_latitude            float64
passenger_count             float64
year                        float64
Month                       float64
Date                        float64
Day                         float64
Hour                        float64
Minute                      float64
dtype: object

In [52]:
#Converting date time to date_time format from object format
train['pickup_datetime'] =  pd.to_datetime(train['pickup_datetime'], format='%Y-%m-%d %H:%M:%S UTC')

In [18]:
train.dtypes

fare_amount                 float64
pickup_datetime      datetime64[ns]
pickup_longitude            float64
pickup_latitude             float64
dropoff_longitude           float64
dropoff_latitude            float64
passenger_count             float64
dtype: object

*There are many rows which have values Na,So we will drop those rows*

In [19]:
train.dropna(subset= ["pickup_datetime"])   #dropping NA values in datetime column

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,4.5,2009-06-15 17:26:21,-73.844311,40.721319,-73.841610,40.712278,1.0
1,16.9,2010-01-05 16:52:16,-74.016048,40.711303,-73.979268,40.782004,1.0
2,5.7,2011-08-18 00:35:00,-73.982738,40.761270,-73.991242,40.750562,2.0
3,7.7,2012-04-21 04:30:42,-73.987130,40.733143,-73.991567,40.758092,1.0
4,5.3,2010-03-09 07:51:00,-73.968095,40.768008,-73.956655,40.783762,1.0
...,...,...,...,...,...,...,...
16062,6.5,2014-12-12 07:41:00,-74.008820,40.718757,-73.998865,40.719987,1.0
16063,16.1,2009-07-13 07:58:00,-73.981310,40.781695,-74.014392,40.715527,2.0
16064,8.5,2009-11-11 11:19:07,-73.972507,40.753417,-73.979577,40.765495,1.0
16065,8.1,2010-05-11 23:53:00,-73.957027,40.765945,-73.981983,40.779560,1.0


 **we will saperate the Pickup_datetime column into separate field like year, month, day of the week, etc**



In [20]:


train['year'] = train['pickup_datetime'].dt.year
train['Month'] = train['pickup_datetime'].dt.month
train['Date'] = train['pickup_datetime'].dt.day
train['Day'] = train['pickup_datetime'].dt.dayofweek
train['Hour'] = train['pickup_datetime'].dt.hour
train['Minute'] = train['pickup_datetime'].dt.minute

In [21]:
train.dtypes

fare_amount                 float64
pickup_datetime      datetime64[ns]
pickup_longitude            float64
pickup_latitude             float64
dropoff_longitude           float64
dropoff_latitude            float64
passenger_count             float64
year                        float64
Month                       float64
Date                        float64
Day                         float64
Hour                        float64
Minute                      float64
dtype: object

In [22]:
test_data['pickup_datetime']= pd.to_datetime(test_data['pickup_datetime'],format= "%Y-%m-%d %H:%M:%S UTC")

In [23]:
test_data.dtypes

pickup_datetime      datetime64[ns]
pickup_longitude            float64
pickup_latitude             float64
dropoff_longitude           float64
dropoff_latitude            float64
passenger_count               int64
dtype: object

**We will seperate pickup date_time column into seperate areas like year,date**


In [24]:
test_data['year'] = test_data['pickup_datetime'].dt.year

In [25]:
test_data.dtypes

pickup_datetime      datetime64[ns]
pickup_longitude            float64
pickup_latitude             float64
dropoff_longitude           float64
dropoff_latitude            float64
passenger_count               int64
year                          int64
dtype: object

In [26]:
test_data['date '] = test_data['pickup_datetime'].dt.date
test_data['month']=test_data['pickup_datetime'].dt.month
test_data['day'] = test_data['pickup_datetime'].dt.dayofweek
test_data['hour'] = test_data['pickup_datetime'].dt.hour
test_data['Minute'] = test_data['pickup_datetime'].dt.minute

In [27]:
test_data.dtypes

pickup_datetime      datetime64[ns]
pickup_longitude            float64
pickup_latitude             float64
dropoff_longitude           float64
dropoff_latitude            float64
passenger_count               int64
year                          int64
date                         object
month                         int64
day                           int64
hour                          int64
Minute                        int64
dtype: object

**Check for the un-necessary data in the train data set**

1>Passenger count should be not more than 6.

2>Longitude should be in the range of -90 to +90.

3>Latitude should be in the range of -180 to +180.

4>Some values which does not seem acceptable,we will remove them.


In [28]:
train['passenger_count'].describe()

count    16012.000000
mean         2.625070
std         60.844122
min          0.000000
25%          1.000000
50%          1.000000
75%          2.000000
max       5345.000000
Name: passenger_count, dtype: float64

**In the above description of the passenger count we see that max number of passenger count is 5435 which is absurd.So we will remove all those row who have value more than 6.**

In [29]:
train = train.drop(train[train["passenger_count"]> 6 ].index, axis=0)

In [30]:
train['passenger_count'].describe()

count    15992.000000
mean         1.643661
std          1.267474
min          0.000000
25%          1.000000
50%          1.000000
75%          2.000000
max          6.000000
Name: passenger_count, dtype: float64

**removing datetime missing values rows.**

In [31]:
train = train.drop(train[train['pickup_datetime'].isnull()].index, axis=0)
print(train.shape)
print(train['pickup_datetime'].isnull().sum())

(16046, 13)
0


**Also removing the values which have the passenger count equal to 0.**

In [32]:
train = train.drop(train[train["passenger_count"] == 0 ].index, axis=0)

In [33]:
train["passenger_count"].describe()

count    15934.000000
mean         1.649581
std          1.265943
min          0.120000
25%          1.000000
50%          1.000000
75%          2.000000
max          6.000000
Name: passenger_count, dtype: float64

In [34]:
train['passenger_count'].sort_values(ascending = True)

8862     0.12
0        1.00
9790     1.00
9791     1.00
9792     1.00
         ... 
8076      NaN
8139      NaN
8259      NaN
8306      NaN
16066     NaN
Name: passenger_count, Length: 15989, dtype: float64

**We observe in sorted passenger count that many values are zero so we will remove them.**

In [35]:
train = train.drop(train[train['passenger_count'].isnull()].index, axis=0)
print(train.shape)
print(train['passenger_count'].isnull().sum())

(15934, 13)
0


One have passenger count 0.12,which is not possible,so we will remove them.

In [36]:

train = train.drop(train[train["passenger_count"] == 0.12 ].index, axis=0)
print(train.shape)
print(train['passenger_count'].isnull().sum())


(15933, 13)
0


**Checking for the longitude and latitude.**




**We will drop all latitude who are not in the range of -90 to +90.**

**We will also drop all longitude who are not in the range of +180 to -180.**


In [37]:
train[train['pickup_latitude']<-90]
train[train['pickup_latitude']>90]

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,Month,Date,Day,Hour,Minute
5686,3.3,2011-07-30 11:15:00,-73.947235,401.083332,-73.951392,40.778927,1.0,2011.0,7.0,30.0,5.0,11.0,15.0


In [38]:
#dropping one value of >90
train = train.drop((train[train['pickup_latitude']<-90]).index, axis=0)
train = train.drop((train[train['pickup_latitude']>90]).index, axis=0)

In [39]:
train[train['pickup_longitude']<-180]
train[train['pickup_longitude']>180]

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,Month,Date,Day,Hour,Minute


In [40]:
train[train['dropoff_latitude']<-90]
train[train['dropoff_latitude']>90]


,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,Month,Date,Day,Hour,Minute


In [41]:
train[train['dropoff_longitude']<-180]
train[train['dropoff_longitude']>180]


,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,Month,Date,Day,Hour,Minute


In [42]:
train['dropoff_longitude'].describe()

count    15932.000000
mean       -72.458781
std         10.587787
min        -74.429332
25%        -73.991182
50%        -73.980156
75%        -73.963607
max         40.802437
Name: dropoff_longitude, dtype: float64

In [43]:
train['dropoff_latitude'].describe()

count    15932.000000
mean        39.895797
std          6.196653
min        -74.006377
25%         40.734655
50%         40.753563
75%         40.768018
max         41.366138
Name: dropoff_latitude, dtype: float64

In [44]:
train['pickup_latitude'].describe()

count    15932.000000
mean        39.895271
std          6.197027
min        -74.006893
25%         40.734937
50%         40.752633
75%         40.767384
max         41.366138
Name: pickup_latitude, dtype: float64

In [45]:
train['dropoff_longitude'].describe()

count    15932.000000
mean       -72.458781
std         10.587787
min        -74.429332
25%        -73.991182
50%        -73.980156
75%        -73.963607
max         40.802437
Name: dropoff_longitude, dtype: float64

**Checking for the fare.**

In [46]:
train['fare_amount'].sort_values(ascending=True)

13032   -3.00
2039    -2.90
2486    -2.50
10002    0.00
2780     0.01
         ... 
1712      NaN
2412      NaN
2458      NaN
8178      NaN
8226      NaN
Name: fare_amount, Length: 15932, dtype: float64

**A lot of fare amount are in negative and many are also null,so we will remove them.**

In [47]:
train = train.drop(train[train['fare_amount'].isnull()].index,axis=0)

In [48]:

train = train.drop(train[train["fare_amount"]<0].index, axis=0)


In [49]:
train = train.drop(train[train["fare_amount"]<1].index, axis=0)
train = train.drop(train[train["fare_amount"]> 454 ].index, axis=0)

In [50]:
train['fare_amount'].describe()

count    15901.000000
mean        11.376864
std         10.815059
min          1.140000
25%          6.000000
50%          8.500000
75%         12.500000
max        453.000000
Name: fare_amount, dtype: float64

**Calculating distance between two points based on the given co-ordinates**

In [58]:
from math import radians, cos, sin, asin, sqrt
def haversine(a):
    longitude1=a[0]
    latitude1=a[1]
    longitude2=a[2]
    latitude2=a[3]
    
    longitude1, latitude1, longitude2, latitude2 = map(radians, [longitude1, latitude1, longitude2, latitude2])
    
    # haversine formula 
    dlong = longitude2 - longitude1 
    dlati = latitude2 - latitude1 
    a = sin(dlati/2)**2 + cos(latitude1) * cos(latitude2) * sin(dlong/2)**2
    c =  2 * asin(sqrt(a))
    # Earth radius = 6,371
    km = 6371* c
    return km

In [59]:
train['distance'] = train[['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude']].apply(haversine,axis=1)

In [62]:
test_data['distance'] = test_data[['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude']].apply(haversine,axis=1)

**Checking the nature of the updated data**

In [64]:
train.head()

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,Month,Date,Day,Hour,Minute,distance
0,4.5,2009-06-15 17:26:21,-73.844311,40.721319,-73.841610,40.712278,1.0,2009.0,6.0,15.0,0.0,17.0,26.0,1.030764
1,16.9,2010-01-05 16:52:16,-74.016048,40.711303,-73.979268,40.782004,1.0,2010.0,1.0,5.0,1.0,16.0,52.0,8.450134
2,5.7,2011-08-18 00:35:00,-73.982738,40.761270,-73.991242,40.750562,2.0,2011.0,8.0,18.0,3.0,0.0,35.0,1.389525
3,7.7,2012-04-21 04:30:42,-73.987130,40.733143,-73.991567,40.758092,1.0,2012.0,4.0,21.0,5.0,4.0,30.0,2.799270
4,5.3,2010-03-09 07:51:00,-73.968095,40.768008,-73.956655,40.783762,1.0,2010.0,3.0,9.0,1.0,7.0,51.0,1.999157


In [65]:
test_data.head()

,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,date,month,day,hour,Minute,distance
0,2015-01-27 13:08:24,-73.973320,40.763805,-73.981430,40.743835,1,2015,2015-01-27,1,1,13,8,2.323259
1,2015-01-27 13:08:24,-73.986862,40.719383,-73.998886,40.739201,1,2015,2015-01-27,1,1,13,8,2.425353
2,2011-10-08 11:53:44,-73.982524,40.751260,-73.979654,40.746139,1,2011,2011-10-08,10,5,11,53,0.618628
3,2012-12-01 21:12:12,-73.981160,40.767807,-73.990448,40.751635,1,2012,2012-12-01,12,5,21,12,1.961033
4,2012-12-01 21:12:12,-73.966046,40.789775,-73.988565,40.744427,1,2012,2012-12-01,12,5,21,12,5.387301


In [66]:
test_data.nunique()

pickup_datetime      1753
pickup_longitude     9124
pickup_latitude      9246
dropoff_longitude    9141
dropoff_latitude     9360
passenger_count         6
year                    7
date                 1211
month                  12
day                     7
hour                   24
Minute                 60
distance             9830
dtype: int64

In [67]:
train.nunique()

fare_amount            459
pickup_datetime      15856
pickup_longitude     13672
pickup_latitude      14110
dropoff_longitude    13763
dropoff_latitude     14136
passenger_count          7
year                     7
Month                   12
Date                    31
Day                      7
Hour                    24
Minute                  60
distance             15448
dtype: int64

In [71]:
train['distance'].sort_values( ascending=False,kind = 'quicksort')

9147     8667.542104
8647     8667.497512
2397     8667.454421
472      8667.304968
11653    8666.701504
            ...     
13050       0.000000
11593       0.000000
2346        0.000000
8331        0.000000
1637        0.000000
Name: distance, Length: 15901, dtype: float64

**In the sorted distance table we see that there are a lot of distance which are zero and those data are not of any use for the prediction.**

**Some values at the top of the table shows that distance travelled are more than 8k km and coming down to the table we get to see that the distance become ariound 127 km, so there is huge diff in the data available, so we will remove them.**

*Removing the zero distance.*

In [75]:
train = train.drop(train[train['distance']==0].index,level = None,axis = 0)

*Removing the distance which are more than 129 kms*


In [76]:
train = train.drop(train[train['distance']>129].index,axis = 0,level = None)

In [77]:
train['distance']

0        1.030764
1        8.450134
2        1.389525
3        2.799270
4        1.999157
           ...   
16061    3.357490
16062    0.850044
16063    7.867638
16064    1.469105
16065    2.590036
Name: distance, Length: 15422, dtype: float64

In [78]:
train.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,Month,Date,Day,Hour,Minute,distance
count,15422.000000,15422.000000,15422.000000,15422.000000,15422.000000,15422.00000,15422.000000,15422.000000,15422.000000,15422.000000,15422.000000,15422.000000,15422.000000
mean,11.372172,-73.910871,40.688668,-73.909890,40.689171,1.65201,2011.737777,6.274348,15.672740,3.035534,13.499741,29.670600,3.426391
std,10.674735,2.679518,2.633422,2.679628,2.633073,1.26771,1.870622,3.447892,8.684236,1.969657,6.510149,17.302001,4.366674
min,1.140000,-74.438233,-74.006893,-74.227047,-74.006377,1.00000,2009.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000111
25%,6.000000,-73.992382,40.736570,-73.991367,40.736307,1.00000,2010.000000,3.000000,8.000000,1.000000,9.000000,15.000000,1.277967
50%,8.500000,-73.982077,40.753357,-73.980568,40.754244,1.00000,2012.000000,6.000000,16.000000,3.000000,14.000000,30.000000,2.190922
75%,12.500000,-73.968158,40.767802,-73.965516,40.768311,2.00000,2013.000000,9.000000,23.000000,5.000000,19.000000,45.000000,3.934563
max,453.000000,40.766125,41.366138,40.802437,41.366138,6.00000,2015.000000,12.000000,31.000000,6.000000,23.000000,59.000000,127.509261
